Querying Force Models at a given time/location

In [1]:
import orekit
from orekit.pyhelpers import setup_orekit_curdir, datetime_to_absolutedate
# orekit.pyhelpers.download_orekit_data_curdir()
vm = orekit.initVM()
setup_orekit_curdir("orekit-data.zip")
#import MU from constants
from org.orekit.utils import Constants
MU = Constants.WGS84_EARTH_MU

from org.orekit.frames import FramesFactory, ITRFVersion
from org.orekit.utils import PVCoordinates
from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.orekit.utils import PVCoordinates, IERSConventions
from org.orekit.orbits import CartesianOrbit
from org.orekit.utils import Constants, PVCoordinatesProvider
from org.orekit.bodies import CelestialBodyFactory
from org.orekit.time import AbsoluteDate, TimeScalesFactory
from org.orekit.forces.gravity.potential import GravityFieldFactory
from org.orekit.forces.gravity import HolmesFeatherstoneAttractionModel, ThirdBodyAttraction, NewtonianAttraction
from org.orekit.propagation import SpacecraftState
from org.orekit.forces import ForceModel

OpenJDK 64-Bit Server VM warning: Attempt to protect stack guard pages failed.
OpenJDK 64-Bit Server VM warning: Attempt to deallocate stack guard pages failed.


In [2]:
SATELLITE_MASS=1000#kg

pv =  [-24166447.8955729 ,  10997956.58418275 ,  2770753.89948468,  -586.37810848 ,-2165.08717971 , 3132.85238624]
# pv = [-13027578.813,   6788658.455,  21921749.703,-1863.07623,-3424.9027  ,  -14.07406]
# pv = [-22873761.86631825,	5701767.971346985,	12830822.295069866,    0,	0,	0]
# pv = [15713.09916*1000,17610.085702*1000,12780.161855*1000,0,0,0]
# pv = [6378137, 0, 0,0,0,0]
print(f"pv:{pv}")

position = Vector3D(float(pv[0]), float(pv[1]), float(pv[2]))
velocity = Vector3D(float(pv[3]), float(pv[4]), float(pv[5]))
pvCoordinates = PVCoordinates(position, velocity)


utc = TimeScalesFactory.getUTC()
# Epoch_date = AbsoluteDate(YYYY, MM, DD, H, M, S, utc)
# Epoch_date = AbsoluteDate(2023, 1, 18, 23, 59, 0.0, utc)
# Epoch_date = AbsoluteDate(2019, 11, 30, 23, 59, 42.0, utc)
Epoch_date = AbsoluteDate(2020, 1, 18, 23, 59, 42.0, utc)

MU = Constants.WGS84_EARTH_MU
monopole_provider = NewtonianAttraction(MU)

gravityProvider = GravityFieldFactory.getNormalizedProvider(0, 0)
gravityfield = HolmesFeatherstoneAttractionModel(FramesFactory.getITRF(IERSConventions.IERS_2010, False), gravityProvider)

moon = CelestialBodyFactory.getMoon()
moon_3dbodyattraction = ThirdBodyAttraction(moon)

sun = CelestialBodyFactory.getSun()
sun_3dbodyattraction = ThirdBodyAttraction(sun)

orbit = CartesianOrbit(pvCoordinates, FramesFactory.getEME2000(), Epoch_date, Constants.WGS84_EARTH_MU)
state = SpacecraftState(orbit, float(SATELLITE_MASS))

monopole_parameters = ForceModel.cast_(monopole_provider).getParameters()
monopole_acc = monopole_provider.acceleration(state, monopole_parameters)

grav_parameters = ForceModel.cast_(gravityfield).getParameters()
grav_acc = gravityfield.acceleration(state, grav_parameters)
grav_pot = gravityfield.value(Epoch_date, position, MU)

moon_3dbody_parameters = ForceModel.cast_(moon_3dbodyattraction).getParameters()
moon_3dbody_acc = moon_3dbodyattraction.acceleration(state, moon_3dbody_parameters)

sun_3dbody_parameters = ForceModel.cast_(sun_3dbodyattraction).getParameters()
sun_3dbody_acc = sun_3dbodyattraction.acceleration(state, sun_3dbody_parameters)

print(f"mono acc {monopole_acc}")
print(f"grav field {grav_acc}")
print(f"grav_pot {grav_pot}")
print(f"moon_3dbody_acc {moon_3dbody_acc}") 
print(f"sun_3dbody_acc {sun_3dbody_acc}")

pv:[-24166447.8955729, 10997956.58418275, 2770753.89948468, -586.37810848, -2165.08717971, 3132.85238624]
mono acc {0.5063336307; -0.2304283738; -0.058052631}
grav field {0; 0; 0}
grav_pot 14931376.108817343
moon_3dbody_acc {0.0000009247; -0.0000025435; -0.000000743}
sun_3dbody_acc {-0.0000002432; 0.0000016901; 0.0000008158}
